In [2]:
import pandas as pd

In [3]:
#generate the filename pattern, specify 2019,2020 as the only year, because the data was large enough to give us nearly 15 million rows of data for only 9 months of 2020 and since tableau public does not support more than that we decided to work only with that. 
date=[]
for i in range(2019,2020):
    for j in range(1,9):
        if j>9:
            d=f'{i}{j}'
            date.append(d)
        else:
            d=f'{i}0{j}'
            date.append(d)

In [4]:
#read all the  available CSVs in a for loop and create a list of frames
frames=[]
for d in date: 
    try:
        filepath=f'RESOURCES/{d}.csv'
        df=pd.read_csv(filepath)
        frames.append(df)
#         print(filepath)
#         df.head()
    except FileNotFoundError:
        pass


In [5]:
#concatenate all the dfs for 2019 and 2020
result = pd.concat(frames)

In [6]:
result.shape
# our dataset has 35 million rows which is not possible to work with in tableau public

(13580306, 15)

In [7]:
#So we decided to look at the station ids which are more popular to choose around 15 million rows.
result["start station id"].value_counts()

519.0     103882
497.0      79474
3255.0     79311
402.0      76831
285.0      73738
           ...  
3432.0       215
3719.0        26
3040.0        10
3250.0         6
3245.0         6
Name: start station id, Length: 830, dtype: int64

In [8]:
# we used the station ids which have been populated more than 80 thousand times in our dataset.
final19=result[result["start station id"].map(result["start station id"].value_counts())>=30000]

In [9]:
final19.dtypes

tripduration                 int64
starttime                   object
stoptime                    object
start station id           float64
start station name          object
start station latitude     float64
start station longitude    float64
end station id             float64
end station name            object
end station latitude       float64
end station longitude      float64
bikeid                       int64
usertype                    object
birth year                   int64
gender                       int64
dtype: object

In [10]:
#so we are keeping less than 15 million rows
final19.shape

(6665910, 15)

In [11]:
#the number of stations we are keeping is 127 out of 1187 but they are most popular 
final19["start station id"].unique().shape

(156,)

In [12]:
final19.sort_values(by=["starttime"],ascending=False)

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
2344223,419,2019-08-31 23:59:59.4520,2019-09-01 00:06:59.0210,447.0,8 Ave & W 52 St,40.763707,-73.985162,529.0,W 42 St & 8 Ave,40.757570,-73.990985,35210,Subscriber,1994,1
2344222,1301,2019-08-31 23:59:58.3620,2019-09-01 00:21:39.7040,3168.0,Central Park West & W 85 St,40.784727,-73.969617,423.0,W 54 St & 9 Ave,40.765849,-73.986905,38664,Customer,1969,0
2344221,1614,2019-08-31 23:59:48.1560,2019-09-01 00:26:42.7760,387.0,Centre St & Chambers St,40.712733,-74.004607,3440.0,Fulton St & Adams St,40.692418,-73.989495,28485,Customer,1969,0
2344218,1767,2019-08-31 23:59:45.7730,2019-09-01 00:29:13.6000,403.0,E 2 St & 2 Ave,40.725029,-73.990697,3141.0,1 Ave & E 68 St,40.765005,-73.958185,25832,Customer,1993,1
2344216,231,2019-08-31 23:59:39.0230,2019-09-01 00:03:30.4820,293.0,Lafayette St & E 8 St,40.730207,-73.991026,285.0,Broadway & E 14 St,40.734546,-73.990741,28225,Subscriber,1993,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9,123,2019-01-01 00:12:19.6030,2019-01-01 00:14:23.1040,432.0,E 7 St & Avenue A,40.726218,-73.983799,3656.0,E 2 St & Avenue A,40.723077,-73.985836,35206,Subscriber,1987,1
7,1547,2019-01-01 00:11:22.9070,2019-01-01 00:37:10.0550,285.0,Broadway & E 14 St,40.734546,-73.990741,3358.0,Garfield Pl & 8 Ave,40.671198,-73.974841,35551,Subscriber,1981,1
4,303,2019-01-01 00:07:35.9450,2019-01-01 00:12:39.5020,229.0,Great Jones St,40.727434,-73.993790,503.0,E 20 St & Park Ave,40.738274,-73.987520,35379,Subscriber,1979,1
3,2719,2019-01-01 00:07:03.5450,2019-01-01 00:52:22.6500,504.0,1 Ave & E 16 St,40.732219,-73.981656,3709.0,W 15 St & 6 Ave,40.738046,-73.996430,21579,Subscriber,1990,1


# longitude and latitude to destination conversion

In [13]:
#     Calculate the great circle distance between two points 
#     on the earth (specified in decimal degrees)

from math import radians, cos, sin, asin, sqrt
def haversine(lon1, lat1, lon2, lat2):
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 3956 # Radius 6371 of earth in kilometers. Use 3956 for miles
    return c * r

In [14]:
# lon1=final["start station longitude"]
# lon2=final["end station longitude"]
# lat1=final["start station latitude"]
# lat2=final["end station latitude"]
new_column = []                    #empty column for distance
for index,row in final19.iterrows():
    lat1 = row["start station latitude"] 
    lon1 = row["start station longitude"] 
    lat2 = row["end station latitude"] 
    lon2 = row["end station longitude"] 
    value = haversine(lat1, lon1, lat2, lon2)  #get the distance
    new_column.append(value)   #append the empty list with distance values

In [15]:
final19.insert(15,"Distance",new_column)  #15 is the index where you want to place your column. Column index starts with 0. "Distance" is the header and new_column are the values in the column.


In [16]:
final19.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,Distance
1,316,2019-01-01 00:04:43.7360,2019-01-01 00:10:00.6080,519.0,Pershing Square North,40.751873,-73.977706,518.0,E 39 St & 2 Ave,40.747804,-73.973442,32723,Subscriber,1964,1,0.304460
3,2719,2019-01-01 00:07:03.5450,2019-01-01 00:52:22.6500,504.0,1 Ave & E 16 St,40.732219,-73.981656,3709.0,W 15 St & 6 Ave,40.738046,-73.996430,21579,Subscriber,1990,1,1.026095
4,303,2019-01-01 00:07:35.9450,2019-01-01 00:12:39.5020,229.0,Great Jones St,40.727434,-73.993790,503.0,E 20 St & Park Ave,40.738274,-73.987520,35379,Subscriber,1979,1,0.479643
7,1547,2019-01-01 00:11:22.9070,2019-01-01 00:37:10.0550,285.0,Broadway & E 14 St,40.734546,-73.990741,3358.0,Garfield Pl & 8 Ave,40.671198,-73.974841,35551,Subscriber,1981,1,1.631488
9,123,2019-01-01 00:12:19.6030,2019-01-01 00:14:23.1040,432.0,E 7 St & Avenue A,40.726218,-73.983799,3656.0,E 2 St & Avenue A,40.723077,-73.985836,35206,Subscriber,1987,1,0.152880


In [17]:
final19.to_csv('Resources/2019-Citibike-data.csv')

In [18]:
final19.Distance.sum()

5305630.449704299

In [19]:
#generate the filename pattern, specify 2020 as the only year, because the data was large enough to give us nearly 15 million rows of data for only 9 months of 2020 and since tableau public does not support more than that we decided to work only with that. 
date=[]
for i in range(2020,2021):
    for j in range(1,9):
        if j>9:
            d=f'{i}{j}'
            date.append(d)
        else:
            d=f'{i}0{j}'
            date.append(d)

In [20]:
#read all the  available CSVs in a for loop and create a list of frames
frames=[]
for d in date: 
    try:
        filepath=f'RESOURCES/{d}.csv'
        df=pd.read_csv(filepath)
        frames.append(df)
#         print(filepath)
#         df.head()
    except FileNotFoundError:
        pass


In [21]:
#concatenate all the dfs for 2019 and 2020
result = pd.concat(frames)

In [22]:
result.shape
# our dataset has 35 million rows which is not possible to work with in tableau public

(11944130, 15)

In [23]:
#So we decided to look at the station ids which are more popular to choose around 15 million rows.
result["start station id"].value_counts()

514     65689
426     64629
3141    58915
435     58123
499     57859
        ...  
4086       31
4087       26
4084       23
3912        3
4090        2
Name: start station id, Length: 1065, dtype: int64

In [91]:
# we used the station ids which have been populated more than 80 thousand times in our dataset.
final20=result[result["start station id"].map(result["start station id"].value_counts())>=23950]

In [92]:
#so we are keeping less than 15 million rows
final20.shape

(5154476, 15)

In [93]:
#the number of stations we are keeping is 127 out of 1187 but they are most popular 
final20["start station id"].unique().shape

(156,)

In [94]:
# lon1=final["start station longitude"]
# lon2=final["end station longitude"]
# lat1=final["start station latitude"]
# lat2=final["end station latitude"]
new_column = []                    #empty column for distance
for index,row in final20.iterrows():
    lat1 = row["start station latitude"] 
    lon1 = row["start station longitude"] 
    lat2 = row["end station latitude"] 
    lon2 = row["end station longitude"] 
    value = haversine(lat1, lon1, lat2, lon2)  #get the distance
    new_column.append(value)   #append the empty list with distance values

In [95]:
final20.insert(15,"Distance",new_column)  #15 is the index where you want to place your column. Column index starts with 0. "Distance" is the header and new_column are the values in the column.


In [96]:
final20.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,Distance
0,789,2020-01-01 00:00:55.3900,2020-01-01 00:14:05.1470,504,1 Ave & E 16 St,40.732219,-73.981656,307,Canal St & Rutgers St,40.714275,-73.989900,30326,Subscriber,1992,1,0.663982
1,1541,2020-01-01 00:01:08.1020,2020-01-01 00:26:49.1780,3423,West Drive & Prospect Park West,40.661063,-73.979453,3300,Prospect Park West & 8 St,40.665147,-73.976376,17105,Customer,1969,1,0.226223
2,1464,2020-01-01 00:01:42.1400,2020-01-01 00:26:07.0110,3687,E 33 St & 1 Ave,40.743227,-73.974498,259,South St & Whitehall St,40.701221,-74.012342,40177,Subscriber,1963,1,2.732620
3,592,2020-01-01 00:01:45.5610,2020-01-01 00:11:38.1550,346,Bank St & Hudson St,40.736529,-74.006180,490,8 Ave & W 33 St,40.751551,-73.993934,27690,Subscriber,1980,1,0.892570
7,1758,2020-01-01 00:02:19.6320,2020-01-01 00:31:37.9640,514,12 Ave & W 40 St,40.760875,-74.002777,426,West St & Chambers St,40.717548,-74.013221,30055,Subscriber,1969,0,1.095103


In [97]:
final20.to_csv('Resources/2020-Citibike-data.csv')

In [98]:
final20.Distance.sum()

4568230.75885815